In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def detect_points(image_path):
    # Read image
    image = cv2.imread(image_path)
    output_image = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    # Poster points detection (top portion)
    poster_roi = gray[50:200, width//4:3*width//4]  # Adjusted ROI for poster area
    _, poster_binary = cv2.threshold(poster_roi, 200, 255, cv2.THRESH_BINARY_INV)
    poster_binary = cv2.dilate(poster_binary, np.ones((3,3), np.uint8), iterations=1)

    poster_contours, _ = cv2.findContours(poster_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    poster_points = []
    for contour in poster_contours:
        area = cv2.contourArea(contour)
        if 10 < area < 150:  # Adjusted area threshold
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"]) + width//4
                cY = int(M["m01"] / M["m00"]) + 50
                poster_points.append((cX, cY))

    # Floor points detection (bottom portion)
    floor_roi = gray[height//2:, :]
    blurred_floor = cv2.GaussianBlur(floor_roi, (5, 5), 0)
    _, floor_binary = cv2.threshold(blurred_floor, 50, 255, cv2.THRESH_BINARY)

    floor_contours, _ = cv2.findContours(floor_binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    floor_points = []
    for contour in floor_contours:
        area = cv2.contourArea(contour)
        if 5 < area < 100:  # Adjusted for floor point size
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"]) + height//2
                floor_points.append((cX, cY))

    # Filter points to keep only the most distinct ones
    def filter_points(points, min_distance=20):
        filtered = []
        points = sorted(points, key=lambda p: (p[1], p[0]))
        for p1 in points:
            if not any(np.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2) < min_distance for p2 in filtered):
                filtered.append(p1)
        return filtered[:15]  # Limit to 15 points

    poster_points = filter_points(poster_points)
    floor_points = filter_points(floor_points)

    # Draw points
    for idx, (x, y) in enumerate(poster_points):
        cv2.circle(output_image, (x, y), 5, (0, 0, 255), -1)
        cv2.putText(output_image, f"P{idx+1}", (x-10, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    for idx, (x, y) in enumerate(floor_points):
        cv2.circle(output_image, (x, y), 5, (0, 255, 0), -1)
        cv2.putText(output_image, f"F{idx+1}", (x-10, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save and display results
    output_path = image_path.replace(".jpg", "_processed.jpg").replace(".png", "_processed.png")
    cv2.imwrite(output_path, output_image)

    print(f"\nFound {len(poster_points)} poster points")
    print(f"Found {len(floor_points)} floor points")
    cv2_imshow(output_image)

    return output_image

# Usage
# img = detect_points('your_image.jpg')
# cv2_imshow(img)  # This will show the image in Colab

In [ ]:
# Example usage
image_path = "/content/poster_A.jpg"
detect_points(image_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Example usage
image_path = "/content/poster_B.jpg"
detect_points(image_path)


Found 10 poster points
Found 15 floor points


array([[[231, 228, 213],
        [231, 228, 213],
        [227, 224, 209],
        ...,
        [177, 179, 173],
        [179, 181, 175],
        [171, 173, 167]],

       [[233, 230, 215],
        [232, 229, 214],
        [228, 225, 210],
        ...,
        [176, 178, 172],
        [180, 182, 176],
        [174, 176, 170]],

       [[226, 223, 208],
        [227, 224, 209],
        [226, 223, 208],
        ...,
        [172, 174, 168],
        [176, 178, 172],
        [171, 173, 167]],

       ...,

       [[ 94, 109, 128],
        [ 91, 106, 125],
        [ 97, 112, 131],
        ...,
        [105, 119, 138],
        [104, 118, 137],
        [105, 119, 138]],

       [[103, 118, 137],
        [ 85, 100, 119],
        [ 87, 102, 121],
        ...,
        [109, 123, 142],
        [111, 125, 144],
        [115, 129, 148]],

       [[110, 125, 144],
        [ 81,  96, 115],
        [ 79,  94, 113],
        ...,
        [102, 116, 135],
        [106, 120, 139],
        [112, 126, 145]]

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def detect_black_points_on_floor(image_path):
    # Load the input image (no resizing)
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get image dimensions
    height, width = gray.shape

    # 1) Define a more aggressive ROI to avoid the top & left side.
    #    Example: bottom 25% of the image, skipping 20% on the left.
    #    Adjust these numbers as needed for your floor layout.
    x1 = int(width * 0.2)   # Skip 20% from the left
    y1 = int(height * 0.75) # Start at 75% from the top
    x2 = width
    y2 = height

    # Extract the ROI
    roi = gray[y1:y2, x1:x2]

    # Apply Gaussian blur to reduce noise (light blur is usually fine)
    blurred = cv2.GaussianBlur(roi, (5, 5), 0)

    # Threshold to isolate dark/black points. Increase if you still detect tape lines.
    _, binary = cv2.threshold(blurred, 80, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # List to store final (x, y) points in full-image coordinates
    coordinates = []

    for contour in contours:
        # Filter by area to exclude tiny noise or huge shapes
        area = cv2.contourArea(contour)
        if 80 < area < 2000:
            M = cv2.moments(contour)
            if M["m00"] != 0:
                # Convert centroid to full-image coordinates by adding ROI offset
                cX = int(M["m10"] / M["m00"]) + x1
                cY = int(M["m01"] / M["m00"]) + y1
                coordinates.append((cX, cY))

    # Sort the coordinates top-to-bottom, left-to-right for consistent labeling
    coordinates.sort(key=lambda point: (point[1], point[0]))

    # Draw only the first 15 points, all in red
    for idx, (x, y) in enumerate(coordinates[:15]):
        # Red circle
        cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
        # Red text
        cv2.putText(image, f"{idx + 1}", (x - 10, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Save the processed image
    output_path = image_path.replace(".png", "_floor_processed.png")
    cv2.imwrite(output_path, image)

    # Print the coordinates
    for idx, (x, y) in enumerate(coordinates[:15]):
        print(f"Point {idx + 1}: ({x}, {y})")

    print(f"Processed image saved at: {output_path}")

    # Display in Colab
    cv2_imshow(image)

# Example usage:
# detect_black_points_on_floor("/content/floor_image.png")


In [ ]:
detect_black_points_on_floor("/content/poster_B.jpg")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def detect_black_points_on_floor(image_path):
    # Load the original image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not read image at", image_path)
        return

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = gray.shape

    # ------------------------------
    # 1) Define your floor ROI here
    #    For example, bottom 30% of the image, skipping 10% from the left & right
    #    Adjust these as needed to focus on the actual floor area.
    # ------------------------------
    x1 = int(width * 0.2)   # Skip 10% from the left
    x2 = int(width * 0.9)   # Skip 10% from the right
    y1 = int(height * 0.8)  # Start at 70% from the top
    y2 = height             # Bottom of the image

    # Extract the floor ROI
    roi = gray[y1:y2, x1:x2]

    # ------------------------------
    # 2) Threshold to isolate dark/black points
    #    Increase if you still detect tape or noise. Decrease if you miss real black dots.
    # ------------------------------
    threshold_value = 80
    _, binary = cv2.threshold(roi, threshold_value, 255, cv2.THRESH_BINARY_INV)

    # ------------------------------
    # 3) Find contours
    # ------------------------------
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Prepare a list for (x, y) floor coordinates
    floor_coords = []

    # ------------------------------
    # 4) Filter contours by area to ignore tiny noise or huge shapes
    # ------------------------------
    min_area = 80
    max_area = 2000

    for contour in contours:
        area = cv2.contourArea(contour)
        if min_area < area < max_area:
            M = cv2.moments(contour)
            if M["m00"] != 0:
                # Convert centroid to full-image coordinates by adding ROI offsets
                cX = int(M["m10"] / M["m00"]) + x1
                cY = int(M["m01"] / M["m00"]) + y1
                floor_coords.append((cX, cY))

    # ------------------------------
    # 5) Sort coordinates (top-to-bottom, then left-to-right)
    # ------------------------------
    floor_coords.sort(key=lambda p: (p[1], p[0]))

    # ------------------------------
    # 6) Select up to 15 points
    # ------------------------------
    selected_points = floor_coords[:15]

    # ------------------------------
    # 7) Draw red circles for those points
    #    No text labels, no green color, no F1, F2, etc.
    # ------------------------------
    for (x, y) in selected_points:
        cv2.circle(image, (x, y), 5, (0, 0, 255), -1)  # Red circle

    # ------------------------------
    # 8) Print their coordinates in the console
    # ------------------------------
    print("Detected floor points (up to 15):")
    for i, (x, y) in enumerate(selected_points, start=1):
        print(f"Point {i}: ({x}, {y})")

    # ------------------------------
    # 9) Save & show the result
    # ------------------------------
    output_path = image_path.replace(".png", "_floor_processed.png")
    cv2.imwrite(output_path, image)
    print(f"\nProcessed image saved at: {output_path}")

    # Show in Colab
    cv2_imshow(image)

# Example usage:
detect_black_points_on_floor("/content/poster_A.jpg")


Output hidden; open in https://colab.research.google.com to view.